# LSH Cascade 検索システム - 最終レポート

## 概要

本プロジェクトでは、大規模ベクトル検索の高速化を目的として、**ITQ-LSH（Iterative Quantization Locality Sensitive Hashing）** と **Overlapセグメントフィルタリング** を組み合わせたカスケード検索システムを開発・評価しました。

### 最終推奨パラメータ

```
埋め込みモデル: intfloat/multilingual-e5-base (768次元)
ITQビット数: 128 bits
Overlap設定: width=8, stride=4 (31セグメント)
```

### 性能サマリー（40万件Wikipedia）

| 指標 | 値 |
|------|----|
| 候補削減率 | 87.5% |
| Recall@10 (limit=1000) | 83.7% |
| ベースラインRecall | 88.8% |
| Recall低下 | -5.1pt |

---

## 実験ノートブック一覧

### Phase 1: モデル・ビット数の評価 (44-46)

| No. | ノートブック | 目的 | 主要な発見 |
|-----|-------------|------|------------|
| 44 | [44_e5_base_bits_evaluation.ipynb](44_e5_base_bits_evaluation.ipynb) | E5-baseでのITQビット数評価 | 64 bitsで99.6% Recall@10達成、ストレージ効率重視なら64 bits推奨 |
| 45 | [45_e5_base_fastembed_compatibility.ipynb](45_e5_base_fastembed_compatibility.ipynb) | FastEmbed互換性検証 | FastEmbedはE5-base未対応、モデル変更時はITQ再学習必要 |
| 46 | [46_fastembed_practical_speed.ipynb](46_fastembed_practical_speed.ipynb) | FastEmbed実用速度評価 | カスタム登録でE5シリーズ使用可能、CPU推論は遅い |

### Phase 2: Overlap評価・最適化 (47-49)

| No. | ノートブック | 目的 | 主要な発見 |
|-----|-------------|------|------------|
| 47 | [47_fastembed_e5base_optimization.ipynb](47_fastembed_e5base_optimization.ipynb) | FastEmbed最適化 + Overlap小規模評価 | 5000件ではOverlapフィルタ効果なし（全件ヒット） |
| 48 | [48_wikipedia_400k_embedding.ipynb](48_wikipedia_400k_embedding.ipynb) | Wikipedia 40万件データセット構築 | E5-base埋め込み + ITQ(64/96/128 bits)学習・永続化 |
| 49 | [49_overlap_evaluation_400k.ipynb](49_overlap_evaluation_400k.ipynb) | 40万件規模Overlap評価 | **128b + Overlap(8,4)** が最適バランス |

### Phase 3: 汎化性能評価 (data/notebooks/)

ResOUデータ（研究報告約2,800件）を用いた汎化性能評価: (データは別途提供)

| No. | ノートブック | 目的 | 主要な発見 |
|-----|-------------|------|------------|
| 01 | data/notebooks/01_prepare_resou_data.ipynb | ResOUデータ準備 | E5-largeで埋め込み生成、DuckDB保存 |
| 02 | data/notebooks/02_itq_generalization_test.ipynb | ITQ汎化テスト | Wikipedia学習ITQの汎化性能検証 |
| 03 | data/notebooks/03_cascade_search_validation.ipynb | カスケード検索検証 | 3段階検索の基本動作確認 |
| 04 | data/notebooks/04_cascade_optimization.ipynb | E5-large最適化 | 外部クエリStep1 R@10=62%（低い） |
| 05 | data/notebooks/05_cascade_e5base_128bits.ipynb | **E5-base最終評価** | 外部クエリStep1 R@10=82%（+20pt改善） |

---

## 主要な技術的発見

### 1. ITQビット数の選択

| ビット数 | Recall@10 (40万件) | ストレージ | 推奨用途 |
|----------|-------------------|-----------|----------|
| 64 bits | 73.8% | 8 bytes | ストレージ重視 |
| 96 bits | 82.5% | 12 bytes | バランス |
| **128 bits** | **88.8%** | 16 bytes | **精度重視（推奨）** |

**結論**: 128 bitsが最も高いRecallを達成。ストレージ増加（+4-8 bytes/doc）は許容範囲。

### 2. Overlapセグメント設定

128 bitsでの各設定の比較:

| 設定 | セグメント数 | 削減率 | Recall@1000 | 評価 |
|------|-------------|--------|-------------|------|
| **(8, 4)** | **31** | **87.5%** | **83.7%** | **推奨** |
| (8, 2) | 61 | 82.2% | 86.0% | 高Recall |
| (12, 6) | 15 | 99.2% | 44.6% | 非推奨 |
| (16, 8) | 11 | 99.9% | 21.1% | 非推奨 |

**結論**: 
- **Overlap(8, 4)** が削減率とRecallの最適バランス
- セグメント幅12bit以上は削減率が高すぎてRecall大幅低下

### 3. E5-base vs E5-large

ResOUデータでの汎化性能比較:

| 指標 | E5-large (04) | E5-base (05) | 改善 |
|------|--------------|--------------|------|
| 外部クエリ Step1 R@10 | 62.0% | **82.0%** | **+20.0pt** |
| 外部クエリ 最終R@10 | 62.0% | **79.5%** | **+17.5pt** |
| 埋め込み次元 | 1024 | 768 | -25% |
| ITQストレージ | 同じ | 同じ | - |

**結論**: E5-baseの方が外部クエリに対する汎化性能が高い。次元数も小さく効率的。

### 4. データ規模の影響

| データ規模 | Overlapフィルタ効果 | 推奨手法 |
|-----------|---------------------|----------|
| ~5,000件 | 効果なし（全件ヒット） | 2段階検索 |
| ~3,000件 | 一部効果あり | 2段階検索 |
| **40万件以上** | **効果大（87%削減）** | **3段階カスケード** |

**結論**: Overlapフィルタは大規模データ（数十万件以上）で真価を発揮。小規模データでは2段階検索（全件ハミング距離→コサイン）が実用的。

---

## 永続化ファイル一覧

### ITQモデル・ハッシュ（data/）

```
data/
├── itq_e5_base_64bits.pkl          # ITQモデル (64 bits)
├── itq_e5_base_96bits.pkl          # ITQモデル (96 bits)
├── itq_e5_base_128bits.pkl         # ITQモデル (128 bits) ★推奨
├── wikipedia_400k_e5_base.duckdb   # 埋め込み + HNSWインデックス (3.4GB)
├── wikipedia_400k_e5_base_embeddings.npy  # 埋め込みベクトル (1.2GB)
├── wikipedia_400k_e5_base_meta.npz        # メタデータ
├── wikipedia_400k_e5_base_hashes_64bits.npy   # ハッシュ (25MB)
├── wikipedia_400k_e5_base_hashes_96bits.npy   # ハッシュ (37MB)
└── wikipedia_400k_e5_base_hashes_128bits.npy  # ハッシュ (49MB) ★推奨
```

### 使用方法

```python
from src.itq_lsh import ITQLSH

# ITQモデルロード
itq = ITQLSH.load('data/itq_e5_base_128bits.pkl')

# 埋め込みからハッシュ生成
hashes = itq.transform(embeddings)  # (n_samples, 128)
```

---

## 3段階カスケード検索アーキテクチャ

```
┌─────────────────────────────────────────────────────────────┐
│                      Query Embedding                        │
│                    (E5-base, 768次元)                       │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  Step 1: Overlap Segment Filter (OR条件)                    │
│  - 128 bits → 31セグメント (8bit幅, 4bitストライド)         │
│  - いずれかのセグメントが一致する文書を候補に               │
│  - 削減率: 87.5% (399k → 50k)                               │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  Step 2: Hamming Distance Sort                              │
│  - 候補文書のハミング距離を計算                             │
│  - 上位N件に絞り込み (例: 1000件)                           │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│  Step 3: Cosine Similarity Ranking                          │
│  - 絞り込んだ候補でコサイン類似度を計算                     │
│  - Top-K (例: 10件) を返却                                  │
└─────────────────────────────────────────────────────────────┘
```

---

## 今後の展望

### 1. 性能改善
- Multi-probe LSH の導入（セグメント近傍も候補に含める）
- ファジーセグメントマッチング（部分一致許容）
- GPU並列化によるハミング距離計算の高速化

### 2. スケーラビリティ
- 分散インデックス対応
- インクリメンタルなITQ更新
- オンラインでのセグメントインデックス更新

### 3. 実運用
- FastAPI/gRPC サービス化
- キャッシュ戦略の最適化
- モニタリング・メトリクス収集

---

## 参考文献

1. **ITQ (Iterative Quantization)**
   - Gong et al., "Iterative Quantization: A Procrustean Approach to Learning Binary Codes", CVPR 2011

2. **E5 Embedding Models**
   - Wang et al., "Text Embeddings by Weakly-Supervised Contrastive Pre-training", 2022
   - [intfloat/multilingual-e5-base](https://huggingface.co/intfloat/multilingual-e5-base)

3. **LSH (Locality Sensitive Hashing)**
   - Indyk & Motwani, "Approximate Nearest Neighbors: Towards Removing the Curse of Dimensionality", STOC 1998